In [1]:
# Tell me what you want to write about
# data = input_group("Let´s Write Something!",[
#    input("What do you want to write about?", name='topic', value="Getting Started With Github"),
#    input("Tell me the first entry of your table of contents", name='toc_seed', value="Introduction" ),
#    select("Select writing style", ['Professional Report', 'University paper','High-school essay','Manual', 'research report'], name='style')])    
    
topic = "mechanical properties of geopolymers"
toc_seed = "Introduction"
style = "University paper"

In [2]:
# Set up libraries
import os
import openai
import json
from dotenv import load_dotenv
from random import choice

load_dotenv()
openai.api_key = os.environ.get('OPENAI_KEY')
completion = openai.Completion()

In [3]:
# Generate a table of contents

intro = topic + "\n1. " + toc_seed + "\n2."

toc_response = openai.Completion.create(
  engine="davinci",
  prompt=intro,
  temperature=0.5,
  max_tokens=70,
  top_p=1,
  frequency_penalty=0.52,
  presence_penalty=0.5,
  stop=["11."]
)

#Uncomment to see raw
#toc_response

In [4]:
# Format the table of contents and get ready for final use. 

TOC=[]
TOC="Title: "+ intro + (toc_response["choices"][0]["text"])
TOC=(TOC.split("\n"))
TOC=[x for x in TOC if x]
TOC

['Title: mechanical properties of geopolymers',
 '1. Introduction',
 '2. Mechanical properties of geopolymers',
 '3. Geopolymer concrete characteristics',
 '4. Applications of geopolymers',
 '5. Properties of geopolymer-based composites',
 '6. Conclusions and future perspectives']

In [5]:
# Manipulate Table Of Contents To Give Input For Writing Stage

followUp=[]
followUp=[e[3:] for e in TOC[1:]]
followUp=["Write a " + style + " on the topic of " +topic +" and "+ str(x) + "\n\"\"\"\"\"\"\n" + str(x)+"\n\"\"\"\"\"\"\nThis is the " + style + " I wrote about " +str(x) + "\n\"\"\"\"\"\"" for x in followUp]

In [6]:
# Generate Output Based On Table Of Contents

content_response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt=followUp,
  temperature=0.5,
  max_tokens=300,
  top_p=1.0,
  frequency_penalty=0.52,
  presence_penalty=0.5,
)

#Uncomment to see raw
content_response

<OpenAIObject text_completion id=cmpl-5AI2pZ6mfCdhAtC7mJNSiIOtspgLB at 0x250950aecc0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nIntroduction\n\nGeopolymers are a relatively new type of material that is made from the reaction of water and a geopolymer material. Geopolymers are typically made from alkaline solutions and silicates, which react to form polysilicate compounds. These polysilicate compounds can be cross-linked with metal oxides such as aluminum oxide, titanium dioxide, or zinc oxide. This paper will discuss the mechanical properties of geopolymers and how they compare to traditional concrete.\n\nMechanical Properties of Geopolymers\n\nThe mechanical properties of geopolymers vary depending on their composition and relative proportions of the different materials used in their production. The properties also depend on what other materials are used in conjunction with them in the production process,

In [7]:
# Output as word document

from docx import Document

document = Document()

document.add_heading(TOC[0],0)

for i in range(0,len(content_response["choices"])):
    document.add_paragraph()
    document.add_heading(TOC[i+1],1)
    document.add_paragraph(content_response["choices"][i]["text"])

document.save(topic + '.docx')

In [8]:
# Output as markdown

f = open(topic + ".md", "a")

f.write("# " + topic)
f.write("\n> This was made using [open.ai](open.ai)")

for i in range(0,len(content_response["choices"])):
    f.write("\n \n")
    f.write("## " + TOC[i+1])
    f.write(content_response["choices"][i]["text"])
    f.write("\n \n")
     
f.close()